![CMCC](http://cmcc.ufabc.edu.br/images/logo_site.jpg)

# **Lab 6 - Fatoração de Matriz para Sistemas de Recomendação**

#### Quando sua base de dados é representada por uma matriz esparsa e sua tarefa é predizer valores faltantes dessa matriz, um algoritmo frequentemente utilizado é a Fatoração de Matriz.

#### Neste notebook vamos utilizar a base de dados de notas de filmes do Movielens em que nos é fornecido uma sequência de tuplas do tipo (usuário, filme, nota) que formam uma matriz esparsa usuários x filmes. O intuito é determinar duas matrizes Usuários x Atributos e Filmes x Atributos que, ao serem multiplicadas, recupera a matriz original.

#### As células-exercícios iniciam com o comentário `# EXERCICIO` e os códigos a serem completados estão marcados pelos comentários `<COMPLETAR>`.

#### ** Nesse notebook: **
#### *Parte 1:* Fatoração da Matriz de Ratings

### **Parte 1: Fatoração da Matriz de Ratings**

#### Cada linha da base do Movielens é representado por id_usuario::id_filme::nota::timestamp, adicionalmente possuímos uma base de dados com informação dos filmes: id_filme::nome::generos.

#### **(1a) Funções de parsing**

#### Para aplicar o algoritmo ALS, visto em aula, precisamos de uma RDD de tuplas do tipo `Rating(usuario,item,nota)`.

#### Como primeira tarefa, complete as funções `parseRatingsRDD` para gerar a tupla de Ratings e `parseMoviesRDD` para gerar uma RDD de tuplas (id_filme, nome) que será utilizada posteriormente.

In [ ]:
# EXERCICIO
import os
import numpy as np
from pyspark.mllib.recommendation import ALS, Rating

def parseRatingsRDD(point):
    """ Parser for the current dataset. It receives a data point and return
        a sentence (third field).
    Args:
        point (str): input data point
    Returns:
        str: a string
    """    
    <COMPLETAR>
    return <COMPLETAR>

def parseMoviesRDD(point):
    data = <COMPLETAR>
    return <COMPLETAR>

filename = os.path.join("Data","Aula06","ratings.dat")
ratingsRDD = (sc.textFile(filename,2)
              .map(parseRatingsRDD)
              )
print 'Exemplo de rating {}'.format(ratingsRDD.take(1)[0])

filename = os.path.join("Data","Aula06","movies.dat")
moviesRDD = (sc.textFile(filename,2)
          .map(parseMoviesRDD)
          )
print 'Exemplo de filme: {}'.format(moviesRDD.take(1)[0])

In [ ]:
assert ratingsRDD.take(1)[0]==Rating(user=1, product=1193, rating=5.0), 'valores incorretos'
print 'ok!'
assert moviesRDD.take(1)[0][1]==u'Toy Story (1995)','valores incorretos'
print 'ok!'

#### **(1b) Geração de um Baseline**

#### Como próximo passo, vamos separar a base de ratings em Treino, Validação e Teste para comparar com um baseline

In [ ]:
weights = [.8, .1, .1]
seed = 42
parsedTrainData, parsedValData, parsedTestData = ratingsRDD.randomSplit(weights, seed)

parsedTrainData.cache()
parsedValData.cache()
parsedTestData.cache()

print 'Traininig examples: {}'.format(parsedTrainData.count())
print 'Validation examples: {}'.format(parsedValData.count())
print 'Test examples: {}'.format(parsedTestData.count())

#### Na célula seguinte complete o código para calcular o baseline para nosso sistema. Primeiro calcule o total de registros na matriz de treino e calcule a nota média dada pelos usuários.

In [ ]:
# EXERCICIO
totalTrain = parsedTrainData.<COMPLETAR>
average = parsedTrainData.<COMPLETAR>

In [ ]:
assert totalTrain==799676, 'valores incorretos'
print 'ok'
assert np.abs(average-3.58228)<1e-5, 'valores incorretos'
print 'ok'

#### Agora, calcule o RMSE para predições utilizando a média na base de validação. Transforme a base de dados de validação para aplicar calcular a média dos erros quadráticos de predição.

In [ ]:
# EXERCICIO
totalVal = parsedValData.<COMPLETAR>

rmseBase = (parsedValData
            .<COMPLETAR>
            .<COMPLETAR>
           )/float(totalVal)

print rmseBase

In [ ]:
assert np.abs(rmseBase-1.253629)<1e-6, 'valores incorretos'
print 'ok'

#### **(1c) Treinando o modelo**

#### Como próximo passo, vamos utilizar a função `ALS` para treinar nossa base de dados, um dos parâmetros de interesse é o rank da matriz decomposta, ela indica quantos atributos latentes o algoritmo irá buscar dentro da matriz original.

#### Utilize um grid search para investigar qual, entre os valores [5, 10, 50] gera o melhor resultado de validação. Não se esqueça de passar o parâmetro seed com valor igual a 42.

In [ ]:
# EXERCICIO
bestRMSE = 10.

def ratings2Tuple(x):
    return ((x.user,x.product),x.rating)

for rank in <COMPLETAR>:
    model = ALS.train(parsedTrainData, rank, seed=seed)
    predictions = model.predictAll( parsedValData.map(lambda x: (x.user,x.product)) )
    trueVsPred = (parsedValData
                  .<COMPLETAR>
                  .<COMPLETAR>
                 )
    rmse = (trueVsPred
            .<COMPLETAR>
            .<COMPLETAR>
           )/float(totalVal)
    print 'RMSE with rank {}: {:.4f}'.format(rank,rmse)
    if rmse < bestRMSE:
        bestModel = model
        bestRMSE = rmse

In [ ]:
assert np.abs(bestRMSE-0.7734)<1e-4,'valores incorretos'
print 'ok'

#### **(1d) Resultados na base teste**

#### Aplique o modelo Baseline e o melhor modelo do [ALS](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.recommendation) na base de testes e compare o resultado.

In [ ]:
# EXERCICIO
totalTest = parsedTestData.<COMPLETAR>

rmseBaseTest = (parsedTestData
                .<COMPLETAR>
                .<COMPLETAR>
               )/float(totalTest)

predictions = bestModel.predictAll(<COMPLETAR>)
trueVsPred = (parsedTestData
              .<COMPLETAR>
              .<COMPLETAR>
             )
rmseALSTest = (trueVsPred
               .<COMPLETAR>
               .<COMPLETAR>
               )/float(totalTest)

print 'Baseline RMSE: {:.4f}, ALS RMSE: {:.4f}'.format(rmseBaseTest,rmseALSTest)

In [ ]:
assert np.allclose(rmseBaseTest,1.2576), 'valores incorretos'
print 'ok'
assert np.abs(rmseALSTest-0.7763)<1e-4, 'valores incorretos'
print 'ok'

In [ ]:
id2movie = moviesRDD.collectAsMap()
countMovies = (predictions
                  .map(lambda x: (id2movie[x.product],1.))
                  .reduceByKey(lambda x,y: x+y)
                 ).collectAsMap()

avgMoviesRDD = (predictions
                .map(lambda x: (id2movie[x.product], x.rating))
                .reduceByKey(lambda x,y: x+y)
                .map(lambda x: (x[0],x[1]/countMovies[x[0]]))
                )
for t in avgMoviesRDD.take(10):
    print 'Avg. predicted rating for {}: {:.2f}'.format(t[0],t[1])